In [4]:
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

import rental_utils; reload(rental_utils)

%matplotlib inline
sns.set(style="whitegrid")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Philipp.Deutsch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_raw = rental_utils.clean(pd.read_json('Data/train.json'))
test_raw = rental_utils.clean(pd.read_json('Data/test.json'))

In [20]:
data = train_test_split(train_raw[['description', 'listing_id']], train_raw['interest_level'], test_size=0.33, random_state=1)
[X_train, X_val, y_train, y_val] = data

In [9]:
train_descriptions = rental_utils.convert_to_words(X_train)

 93% (30973 of 33065) |##################  | Elapsed Time: 0:01:10 ETA: 0:00:04C:\Users\Philipp.Deutsch\AppData\Local\Continuum\Anaconda2\envs\ds\lib\site-packages\bs4\__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
 99% (33057 of 33065) |################### | Elapsed Time: 0:01:15 ETA: 0:00:00


In [10]:
val_descriptions = rental_utils.convert_to_words(X_val)

100% (16287 of 16287) |###################| Elapsed Time: 0:00:35 Time: 0:00:35


In [11]:
test_descriptions = rental_utils.convert_to_words(test_raw)

100% (74659 of 74659) |###################| Elapsed Time: 0:02:53 Time: 0:02:53


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word",
                                 tokenizer=None,
                                 preprocessor=None,
                                 stop_words=None,
                                 max_features=1000)
train_data_features = vectorizer.fit_transform(train_descriptions).toarray()
val_data_features = vectorizer.transform(val_descriptions).toarray()

In [27]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit(train_data_features, y_train)

pred_val = forest.predict(val_data_features)
ct = pd.crosstab(pred_val, y_val, margins=False)
print(ct)

interest_level  high    low  medium
row_0                              
high             161     96     187
low              856  10558    2877
medium           237    616     699


In [16]:
test_data_features = vectorizer.transform(test_descriptions).toarray()
pred_train = forest.predict(train_data_features)
pred_test = forest.predict(test_data_features)

In [28]:
df_train = pd.DataFrame({'listing_id':X_train['listing_id'], 'desc_pred':pred_train})
df_val = pd.DataFrame({'listing_id':X_val['listing_id'], 'desc_pred':pred_val})

df_train = pd.concat([df_train, df_val])
df_test = pd.DataFrame({'listing_id':test_raw['listing_id'], 'desc_pred':pred_test})

print(df_train.shape)
print(df_test.shape)

(49352, 2)
(74659, 2)


In [29]:
df_train.to_csv('Data/description_prediction_train.csv', index=False)
df_test.to_csv('Data/description_prediction_test.csv', index=False)